# scrapper

In [ ]:
cl="moduleFade__Frame-sc-53f9yi-0 eCGjpA"
import requests 
from bs4 import BeautifulSoup 

url =  "https://www.coinbase.com/price/stellar"
r = requests.get(url) 

soup = BeautifulSoup(r.content,
                    ) 


In [ ]:
a = soup.find('div',attrs={'class':cl})

In [ ]:
[x.text for x in soup.find_all('article')]

In [ ]:
# Import libraries
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# NLTK VADER for sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer

finwiz_url = 'https://finviz.com/quote.ashx?t='


In [ ]:
news_tables = {}
tickers = ['AMZN', 'TSLA', 'GOOG',]

for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
    response = urlopen(req)    
    # Read the contents of the file into 'html'
    html = BeautifulSoup(response)
    # Find 'news-table' in the Soup and load it into 'news_table'
    news_table = html.find(id='news-table')
    # Add the table to our dictionary
    news_tables[ticker] = news_table

In [ ]:
# Read one single day of headlines for 'AMZN' 
amzn = news_tables['AMZN']
# Get all the table rows tagged in HTML with <tr> into 'amzn_tr'
amzn_tr = amzn.findAll('tr')

for i, table_row in enumerate(amzn_tr):
    # Read the text of the element 'a' into 'link_text'
    a_text = table_row.a.text
    # Read the text of the element 'td' into 'data_text'
    td_text = table_row.td.text
    # Print the contents of 'link_text' and 'data_text' 
    print(a_text)
    print(td_text)
    # Exit after printing 4 rows of data
    if i == 3:
        break

In [ ]:
parsed_news = []

# Iterate through the news
for file_name, news_table in news_tables.items():
    # Iterate through all tr tags in 'news_table'
    for x in news_table.findAll('tr'):
        # read the text from each tr tag into text
        # get text from a only
        text = x.a.get_text() 
        # splite text in the td tag into a list 
        date_scrape = x.td.text.split()
        # if the length of 'date_scrape' is 1, load 'time' as the only element

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        # else load 'date' as the 1st element and 'time' as the second    
        else:
            date = date_scrape[0]
            time = date_scrape[1]
        # Extract the ticker from the file name, get the string up to the 1st '_'  
        ticker = file_name.split('_')[0]
        
        # Append ticker, date, time and headline as a list to the 'parsed_news' list
        parsed_news.append([ticker, date, time, text])
        
parsed_news

In [ ]:
# Instantiate the sentiment intensity analyzer
vader = SentimentIntensityAnalyzer()

# Set column names
columns = ['ticker', 'date', 'time', 'headline']

# Convert the parsed_news list into a DataFrame called 'parsed_and_scored_news'
parsed_and_scored_news = pd.DataFrame(parsed_news, columns=columns)

# Iterate through the headlines and get the polarity scores using vader
scores = parsed_and_scored_news['headline'].apply(vader.polarity_scores).tolist()

# Convert the 'scores' list of dicts into a DataFrame
scores_df = pd.DataFrame(scores)

# Join the DataFrames of the news and the list of dicts
parsed_and_scored_news = parsed_and_scored_news.join(scores_df, rsuffix='_right')

# Convert the date column from string to datetime
parsed_and_scored_news['date'] = pd.to_datetime(parsed_and_scored_news.date).dt.date

parsed_and_scored_news.head()

In [ ]:
from flair.models import TextClassifier

from flair.data import Sentence

classifier = TextClassifier.load('en-sentiment')

sentence = Sentence('Amazon breaks down in Brazil, riding e-commerce boom set off by COVID-19 distancing')

classifier.predict(sentence)

# print sentence with predicted labels
print('Sentence above is: ', sentence.labels)

In [ ]:
sentence.labels[0].score

In [ ]:
def f(x):
    sentence = Sentence(x)
    classifier.predict(sentence)
    sign = -1 if sentence.labels[0].value == 'NEGATIVE' else 1
    score = sentence.labels[0].score*sign
    return score

In [ ]:
parsed_and_scored_news["flair"] = parsed_and_scored_news['headline'].map(f )

In [ ]:
parsed_and_scored_news

In [ ]:
import cufflinks as cf
cf.go_offline()

In [ ]:

plt.rcParams['figure.figsize'] = [10, 6]

# Group by date and ticker columns from scored_news and calculate the mean
mean_scores = parsed_and_scored_news.groupby(['ticker','date']).mean()

# Unstack the column ticker
mean_scores = mean_scores.unstack()

# Get the cross-section of compound in the 'columns' axis
mean_scores = mean_scores.xs('compound', axis="columns").transpose()

# Plot a bar chart with pandas
mean_scores.iplot(kind = 'bar')

# News API


In [ ]:
token = "f4f85aabb9434483b2d05201e573f3eb"

In [ ]:
pip install newsapi-python


In [ ]:
from newsapi import NewsApiClient

# Init
newsapi = NewsApiClient(api_key=token)

In [ ]:
# # /v2/top-headlines
# top_headlines = newsapi.get_top_headlines(q='bitcoin',
#                                           sources='bbc-news,the-verge',
#                                           category='business',
#                                           language='en',
#                                           country='us')

# /v2/everything
all_articles = newsapi.get_everything(q='bitcoin',
                                      sources='bbc-news,the-verge',
                                      domains='bbc.co.uk,techcrunch.com',
                                      from_param='2017-12-01',
                                      to='2017-12-12',
                                      language='en',
                                      sort_by='relevancy',
                                      page=2)

# /v2/sources
sources = newsapi.get_sources()

# Google news

In [ ]:
import pandas as pd
from GoogleNews import GoogleNews
#only 10 per search

### Get Dates

In [188]:
first = pd.to_datetime("01/01/2018",dayfirst= True,)
last = pd.to_datetime("09/11/2020",dayfirst= True,)
all_dates = pd.date_range(first,last,freq='d').to_series().dt.strftime("%m/%d/%Y").values#Format of dates day/month/year

In [189]:
all_dates


array(['01/01/2018', '01/02/2018', '01/03/2018', ..., '11/07/2020',
       '11/08/2020', '11/09/2020'], dtype=object)

In [159]:
googlenews = GoogleNews(lang='en',period='d') #today
googlenews = GoogleNews(lang='en',start=all_dates[0],end=all_dates[0])#Format of dates month/day/year

In [160]:
googlenews.search('Nasdaq')

In [161]:
result= (googlenews.result())

In [165]:
googlenews.getpage(2)

In [162]:
for i in range(3,7):
    googlenews.getpage(i)
    result=googlenews.result()
    df=pd.DataFrame(result)

In [163]:
df.drop_duplicates(subset='link')

,title,media,date,desc,link,img
0,Union Bankshares Corporation Completes Acquisi...,,1 ene. 2018,"Headquartered in Richmond, Virginia, Union Ban...",https://www.globenewswire.com/news-release/201...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
1,"4 Preferred Funds Paying Up to 7.2% – 2 Buys, ...",Nasdaq,1 ene. 2018,"Nasdaq 100 Movers: BIIB, QCOM. Contributor. BN...",https://www.nasdaq.com/articles/4-preferred-fu...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
2,Steady Hand Leads Check Point to New Post Dot-...,CTech,1 ene. 2018,The Nasdaq-listed cybersecurity company recent...,"https://www.calcalistech.com/ctech/articles/0,...","data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
3,The Social Security Administration May Be Stee...,Nasdaq,1 ene. 2018,Sessions College President Gordon Drummond joi...,https://www.nasdaq.com/articles/social-securit...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
4,Calculating The Fair Value Of QVC Group (NASDA...,,1 ene. 2018,Calculating The Fair Value Of QVC Group (NASDA...,https://simplywall.st/stocks/us/retail/nasdaq-...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
5,National Commerce Corporation Announces Closin...,,1 ene. 2018,"01, 2018 (GLOBE NEWSWIRE) -- National Commerce...",https://www.globenewswire.com/news-release/201...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
6,PepsiCo's Stock Buybacks: A Mixed Record In Ge...,Seeking Alpha,1 ene. 2018,The purpose of the article is to consider the ...,https://seekingalpha.com/article/4134688-pepsi...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
7,Overstock: The Perfect Stock Blockchain Play,Seeking Alpha,1 ene. 2018,"Peloton: Wait For The Crash, Then Buy The Stoc...",https://seekingalpha.com/article/4134692-overs...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
8,Ripple was the best-performing cryptocurrency ...,Quartz,1 ene. 2018,"After all, the Nasdaq Composite Index gained 2...",https://qz.com/1169000/ripple-was-the-best-per...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
9,How high-frequency trading hit a speed bump,Financial Times,1 ene. 2018,Plugging into exchanges such as the NYSE and N...,https://www.ft.com/content/d81f96ea-d43c-11e7-...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."


In [194]:
def get_data_news(dates,topic,pages,lang):
    result = []
    
    for i in dates:
        googlenews = GoogleNews(lang=lang,start=i,end=i)
        googlenews.search(topic)
        for p in range(2,pages+1):
            googlenews.getpage(p)
        r = googlenews.result()
#         pd.to_pickle(r,f"google_news_{topic}_{i}.pkl")
        result.extend(r)
        pd.to_pickle(result,f"google_news_{topic}_{dates[0]}.pkl")
        
#         df.append(pd.DataFrame(result))
    df=pd.DataFrame(result)
    df.drop_duplicates(subset='link',inplace=True)
    return df

In [195]:
 data = get_data_news(dates=all_dates[-2:],topic='nasdaq',pages=2,lang='en')

In [196]:
data

,title,media,date,desc,link,img
0,Inseego Corp. (NASDAQ:INSG) Just Released Its ...,Yahoo Finance,hace 1 día,(NASDAQ:INSG) Just Released Its Third-Quarter ...,https://finance.yahoo.com/news/inseego-corp-na...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
1,"Mesa Laboratories, Inc. Just Beat Earnings Exp...",,hace 1 día,(NASDAQ:MLAB) just released its latest second-...,https://www.nasdaq.com/articles/mesa-laborator...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
2,Inseego Corp. (NASDAQ:INSG) Third-Quarter Resu...,,hace 1 día,(NASDAQ:INSG) Third-Quarter Results: Here's Wh...,https://simplywall.st/stocks/us/tech/nasdaq-in...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
3,"GLOBAL MARKETS-Risk assets cheer Biden win, do...",Nasdaq,hace 1 día,E-mini futures for the S&P 500 ESc1 jumped mor...,https://www.nasdaq.com/articles/global-markets...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
4,3 Things to Watch in the Stock Market This Week,,hace 1 día,A few widely followed companies will announce ...,https://www.nasdaq.com/articles/3-things-to-wa...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
5,"Acorda Therapeutics, Inc. (NASDAQ:ACOR) Analys...",Yahoo Finance,hace 1 día,(NASDAQ:ACOR) Analysts Are Reducing Their Fore...,https://finance.yahoo.com/news/acorda-therapeu...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
6,"The Dow Jones, SP 500 and Nasdaq rose on a wee...",The Washington Newsday,hace 1 día,"This week the Dow Jones rose 6.9%, the S&P 500...",https://washingtonnewsday.com/news/the-dow-jon...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
7,Bearish: Analysts Just Cut Their MannKind Corp...,Yahoo Finance,hace 1 día,Bearish: Analysts Just Cut Their MannKind Corp...,https://finance.yahoo.com/news/bearish-analyst...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
8,"G1 Therapeutics, Inc. (NASDAQ:GTHX) Third-Quar...",Yahoo Finance,hace 1 día,(NASDAQ:GTHX) Third-Quarter Results: Here's Wh...,https://finance.yahoo.com/news/g1-therapeutics...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
9,ANALYSIS-Russia and Turkey keep powder dry in ...,Nasdaq,hace 22 horas,Nasdaq Chief Economist Phil Mackintosh joins G...,https://www.nasdaq.com/articles/analysis-russi...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."


In [197]:
len(data)

40